In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [2]:
#conda create --name py38 python=3.8

In [2]:
pip install tabula-py

Note: you may need to restart the kernel to use updated packages.


In [3]:
from tabula import read_pdf
import tabula 

In [4]:
url="https://www.regionh.dk/english/about-the-capital-region/facts-about-the-region/PublishingImages/PostalcodesEnglish.pdf"
tabula.convert_into(url, "output.csv", pages='all', output_format="csv")



In [31]:
df3=pd.read_csv('output.csv')
df3.drop(columns='Company', inplace = True)
df3.drop(index=[0,3,4,5,6], inplace = True)
df3=df3[df3.City.str[:] != "Klemensker"]
df3=df3[df3.City.str[:] != "Ølstykke"]
df3=df3[df3.City.str[:] != "Jægerspris"]
df3=df3[df3.City.str[:] != "Frederikssund"]
df3=df3[df3.City.str[:] != "Stenløse"]
df3=df3[df3.City.str[:] != "Veksø Sjælland"]
df3=df3[df3.City.str[:] != "Nexø"]
df3=df3[df3.City.str[:] != "Svaneke"]
df3=df3[df3.City.str[:] != "Rønne"]
df3=df3[df3.City.str[:] != "Aakirkeby"]
df3=df3[df3.City.str[:] != "Østermarie"]
df3=df3[df3.City.str[:] != "Hasle"]
df3=df3[df3.City.str[:] != "GudhjemHasle"]
df3=df3[df3.City.str[:] != "Allinge"]
df3=df3[df3.City.str[:] != "Gudhjem"]
df3=df3[df3.Street.str[:] != "Klostergården"]
df3=df3[df3.Street.str[:] != "Christians Brygge ulige nr. + 2-22"]
df3=df3[df3.Street.str[:] != "Rørholmsgade"]
df3 = df3.replace(np.nan, '', regex=True)
df3=df3.replace({'Postboks': ''})
df3=df3.replace({'Ufrankerede svarforsendelser': ''})
df3.iloc[0,0]=2630
df3.iloc[1,0]=2500
df3.iloc[7,0]=2300
df3['District']=df3['Street']+", "+df3['City']
df=df3.reset_index(drop=True)
#df=df.groupby('District').mean().astype(int)
df.shape

(721, 4)

In [ ]:
for i in range(df.shape[0]):
    while df.iloc[i,3].startswith(', '):
        df.groupby('District').mean()

let's find out the geograpical coordinate of Copenhagen 

In [6]:
geolocator = Nominatim(user_agent="Cph_explorer")
location = geolocator.geocode('Copenhagen, Denmark')
ltd = location.latitude
lntd = location.longitude
print('The geograpical coordinate of Copenhagen, the capital of Denmark, are {}, {}.'.format(ltd, lntd))

The geograpical coordinate of Copenhagen, the capital of Denmark, are 55.6867243, 12.5700724.


to find geographical coordinates of each postal code, I use pgeocode library 


In [7]:
pip install pgeocode

Note: you may need to restart the kernel to use updated packages.


In [8]:
newdf= pd.DataFrame( columns=['Latitude', 'Longitude'])

In [9]:
geolocator = Nominatim(user_agent="Cph_explorer")
import pgeocode
nomi = pgeocode.Nominatim('dk')

for i in range(df.shape[0]):
    n2=i
    if df.iloc[i,3].startswith(', '):
        location = nomi.query_postal_code(int(df.iloc[i,0]))
        newdf.loc[i,'Latitude'] = location.latitude
        newdf.loc[i,'Longitude']= location.longitude
        
    else:
        location = geolocator.geocode(df.iloc[i,3])
        newdf.loc[i,'Latitude'] = location.latitude
        newdf.loc[i,'Longitude']= location.longitude
        print(df.iloc[i,3])
    

Girostrøget 1, Høje Taastrup
Vigerslev Allé 18, Valby
Emil Holms Kanal 20, København C
Købmagergade 33, København K
Kongens Nytorv, København K
Nyhavn, København K
Herluf Trolles Gade, København K
Cort Adelers Gade, København K
Peder Skrams Gade, København K
Tordenskjoldsgade, København K
August Bournonvilles Passage, København K
Heibergsgade, København K
Holbergsgade, København K
Havnegade, København K
Niels Juels Gade, København K
Holmens Kanal, København K
Ved Stranden, København K
Boldhusgade, København K
Laksegade, København K
Asylgade, København K
Fortunstræde, København K
Admiralgade, København K
Nikolaj Plads, København K
Nikolajgade, København K
Bremerholm, København K
Vingårdstræde, København K
Dybensgade, København K
Lille Kirkestræde, København K
Store Kirkestræde, København K
Lille Kongensgade, København K
Holmens Kanal 2-12, København K
Havnegade 5, København K
Kongens Nytorv 13, København K
Esplanaden 50, København K
Østergade, København K
Ny Østergade, København K
Pisto

GeocoderTimedOut: Service timed out

In [ ]:
#i=174
df.to_pickle(progress) 

In [10]:

df_new2=pd.concat([df,newdf], axis=1)
for i in range(df_new2.shape[0]):
    if df_new2.iloc[i,3].startswith(', '):
        df_new2.iloc[i,3]=df_new2.iloc[i,3].replace(', ','')
        
df_new2=df_new2.dropna(axis=0)
df_new2.head()

,Postal Code,City,Street,District,Latitude,Longitude
0,2630,Høje Taastrup,Girostrøget 1,"Girostrøget 1, Høje Taastrup",55.6479,12.275
1,2500,Valby,Vigerslev Allé 18,"Vigerslev Allé 18, Valby",55.6624,12.5212
2,999,København C,Emil Holms Kanal 20,"Emil Holms Kanal 20, København C",55.6583,12.5903
3,1000,København K,Købmagergade 33,"Købmagergade 33, København K",55.6804,12.5781
4,1001,København K,,København K,55.6759,12.5655


In [11]:
df_new2

,Postal Code,City,Street,District,Latitude,Longitude
0,2630,Høje Taastrup,Girostrøget 1,"Girostrøget 1, Høje Taastrup",55.6479,12.275
1,2500,Valby,Vigerslev Allé 18,"Vigerslev Allé 18, Valby",55.6624,12.5212
2,999,København C,Emil Holms Kanal 20,"Emil Holms Kanal 20, København C",55.6583,12.5903
3,1000,København K,Købmagergade 33,"Købmagergade 33, København K",55.6804,12.5781
4,1001,København K,,København K,55.6759,12.5655
5,1002,København K,,København K,55.6759,12.5655
6,1003,København K,,København K,55.6759,12.5655
7,2300,København K,,København K,55.6759,12.5655
8,1005,København K,,København K,55.6759,12.5655
9,1006,København K,,København K,55.6759,12.5655


**IT WORKS **

#### Create a map of Copenhagen with neighborhoods superimposed on top.

In [73]:
# create map of Copenhagen using latitude and longitude values
map_CPH = folium.Map(location=[ltd, lntd], zoom_start=9)

# add markers to map
for lat, lng, District, PC in zip(df_new2['Latitude'], df_new2['Longitude'], df_new2['District'], df_new2['Postal Code']):
    label = '{}, {}'.format(District, PC)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_CPH)  
    
map_CPH

However, for illustration purposes, let's simplify the above map and segment and cluster only the boroughs that contains Toronto in its name. So let's slice the original dataframe and create a new dataframe.

In [37]:
Downtown_data = df_new2[df_new2['Postal Code']<3000].reset_index(drop=True)
Downtown_data



,Postal Code,City,Street,District,Latitude,Longitude
0,2630,Høje Taastrup,Girostrøget 1,"Girostrøget 1, Høje Taastrup",55.6479,12.275
1,2500,Valby,Vigerslev Allé 18,"Vigerslev Allé 18, Valby",55.6624,12.5212
2,999,København C,Emil Holms Kanal 20,"Emil Holms Kanal 20, København C",55.6583,12.5903
3,1000,København K,Købmagergade 33,"Købmagergade 33, København K",55.6804,12.5781
4,1001,København K,,København K,55.6759,12.5655
5,1002,København K,,København K,55.6759,12.5655
6,1003,København K,,København K,55.6759,12.5655
7,2300,København K,,København K,55.6759,12.5655
8,1005,København K,,København K,55.6759,12.5655
9,1006,København K,,København K,55.6759,12.5655


Let's get the geographical coordinates of Downtown Toronto .

In [38]:
geolocator = Nominatim(user_agent="Cph_explorer")
location = geolocator.geocode('Frederiksberg, Copenhagen')
ltd2 = location.latitude
lntd2 = location.longitude
print('The geograpical coordinate of Frederiksberg are {}, {}.'.format(ltd2, lntd2))


The geograpical coordinate of Frederiksberg are 55.678016, 12.5326186.


As we did with all of Toronto, let's visualiza Downtown Toronto and the neighborhoods in it.

In [65]:

# create map of Toronto using latitude and longitude values
map_Downtown = folium.Map(location=[ltd2, lntd2], zoom_start=10)

# add markers to map
for lat, lng, District, PC in zip(Downtown_data['Latitude'], Downtown_data['Longitude'], Downtown_data['District'], Downtown_data['Postal Code']):
    label = '{}, {}'.format(District, PC)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Downtown)  
    
map_Downtown

**FourSqaure Credentials**


In [40]:
CLIENT_ID = 'SIBIHQFDK1SAYYZMIIFIKUHTBJOSRHSK1E42CBECAN3QGQLD' 
CLIENT_SECRET = 'B2LOXLDUNL4T1FBIUYEFTH5QAT22YSFF1MYA1NQWGDHOI4DV' 
VERSION = '20180605' 

Let's explore the first neighborhood in our dataframe.

In [42]:
neighborhood_latitude = Downtown_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Downtown_data.loc[0, 'Longitude'] # neighborhood longitude value
neighborhood_name = Downtown_data.loc[0, 'District'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Girostrøget 1, Høje Taastrup are 55.647894, 12.275044.


#### Now, let's get the top 100 venues that are in Regent Park within a radius of 500 meters.

In [49]:
# type your answer here
LIMIT = 100 
radius = 1000

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, radius, LIMIT)
url



'https://api.foursquare.com/v2/venues/explore?client_id=SIBIHQFDK1SAYYZMIIFIKUHTBJOSRHSK1E42CBECAN3QGQLD&client_secret=B2LOXLDUNL4T1FBIUYEFTH5QAT22YSFF1MYA1NQWGDHOI4DV&ll=55.647894,12.275044&v=20180605&radius=1000&limit=100'

From the Foursquare lab in the previous module, we know that all the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [50]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [51]:
results = requests.get(url).json()
results


{'meta': {'code': 200, 'requestId': '5ec45608a2e538001b14dd4a'},
 'response': {'headerLocation': 'Vestegnen',
  'headerFullLocation': 'Vestegnen, Copenhagen',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 19,
  'suggestedBounds': {'ne': {'lat': 55.65689400900001,
    'lng': 12.290963847106793},
   'sw': {'lat': 55.63889399099999, 'lng': 12.259124152893206}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ece12c08b81b47393db4725',
       'name': 'Café Lett',
       'location': {'address': 'Banestrøget 5',
        'lat': 55.64869861883249,
        'lng': 12.269478346261025,
        'labeledLatLngs': [{'label': 'display',
          'lat': 55.64869861883249,
          'lng': 12.269478346261025}],
        'distance': 360,
        'postalCode': '2630',
   

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [52]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/Users/anasrezk/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Café Lett,Café,55.648699,12.269478
1,Nike Factory Store,Sporting Goods Shop,55.644533,12.279653
2,CAFÉ A,Café,55.643216,12.277520
3,Cafe Crema,Café,55.643767,12.277197
4,Føtex City 2,Grocery Store,55.643848,12.274701


And how many venues were returned by Foursquare?

In [53]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

19 venues were returned by Foursquare.


## 2. Explore Neighborhoods in Downtown Toronto

Let's create a function to repeat the same process to all the neighborhoods in Downtown Toronto

In [54]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [56]:
# type your answer here

Downtown_venues = getNearbyVenues(names=Downtown_data['District'],
                                   latitudes=Downtown_data['Latitude'],
                                   longitudes=Downtown_data['Longitude']
                                  )




Girostrøget 1, Høje Taastrup
Vigerslev Allé 18, Valby
Emil Holms Kanal 20, København C
Købmagergade 33, København K
København K
København K
København K
København K
København K
København K
København K
København K
København K
København K
København K


KeyError: 'groups'

In [ ]:
Downtown_venues.head()

Let's check how many venues were returned for each neighborhood

In [ ]:
Downtown_venues.groupby('Neighborhood').count()

 Let's find out how many unique categories can be curated from all the returned venues


In [ ]:
print('There are {} uniques categories.'.format(len(Downtown_venues['Venue Category'].unique())))

## 3. Analyze Each Neighborhood

In [ ]:
# one hot encoding
Down_onehot = pd.get_dummies(Downtown_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Down_onehot['Neighborhood'] = Downtown_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Down_onehot.columns[-1]] + list(Down_onehot.columns[:-1])
Down_onehot = Down_onehot[fixed_columns]

Down_onehot.head()

 Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [ ]:
DOwn_grouped = Down_onehot.groupby('Neighborhood').mean().reset_index()
DOwn_grouped

#### Let's print each neighborhood along with the top 5 most common venues

In [ ]:
num_top_venues = 5

for hood in DOwn_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = DOwn_grouped[DOwn_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

First, let's write a function to sort the venues in descending order.

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = DOwn_grouped['Neighborhood']

for ind in np.arange(DOwn_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(DOwn_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [ ]:
# set number of clusters
kclusters = 7

Downtown_grouped_clustering = DOwn_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Downtown_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Down_merged = Downtown_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Down_merged = Down_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Down_merged.head() # check the last columns!

Finally, let's visualize the resulting clusters

In [ ]:
# create map
map_clusters = folium.Map(location=[ltd2, lntd2], zoom_start=11)


# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Down_merged['Latitude'], Down_merged['Longitude'], Down_merged['Neighborhood'], Down_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Examine clusters:

In [ ]:
Down_merged.loc[Down_merged['Cluster Labels'] == 0, Down_merged.columns[[1] + list(range(5, Down_merged.shape[1]))]]